# Dense Neural Network Regression  

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to approximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [84]:
import pandas as pd

In [85]:
housedata = pd.read_csv('cal_housing_clean.csv')

In [86]:
housedata.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [87]:
housedata.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [88]:
from sklearn.model_selection import train_test_split

In [89]:
X = housedata.drop('medianHouseValue',axis = 1)
y = housedata['medianHouseValue']


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [91]:
from sklearn.preprocessing import MinMaxScaler

In [92]:
scaler = MinMaxScaler()

In [93]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [94]:
X_train = pd.DataFrame(data = scaler.transform(X_train),columns=X_train.columns,index=X_train.index)

In [95]:
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns,index=X_test.index)

In [96]:
X_test.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16086,0.686275,0.046264,0.045158,0.025873,0.048841,0.353133
8816,0.705882,0.027417,0.020795,0.012709,0.023187,0.770182
7175,0.901961,0.032326,0.040813,0.041662,0.042592,0.133626
16714,0.313725,0.043212,0.046089,0.032840,0.048018,0.263576
14491,0.411765,0.088433,0.069367,0.043728,0.072192,0.660046


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [97]:
import tensorflow as tf

In [98]:
X_train.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome'],
      dtype='object')

In [99]:
housingMedianAge = tf.feature_column.numeric_column('housingMedianAge') 
totalRooms = tf.feature_column.numeric_column('totalRooms') 
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms') 
population = tf.feature_column.numeric_column('population') 
households = tf.feature_column.numeric_column('households') 
medianIncome = tf.feature_column.numeric_column('medianIncome') 

In [100]:
feat_cols = [housingMedianAge, totalRooms, totalBedrooms, population,
       households, medianIncome]

** Create the input function for the estimator object.**

In [101]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train,y_train,batch_size= 10,num_epochs = 1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [102]:
model = tf.estimator.DNNRegressor([6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\Surbhi\\AppData\\Local\\Temp\\tmpca_tpb35', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_steps': None}


##### ** Train the model for ~1,000 steps.

In [120]:
model.train(input_func,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\Surbhi\AppData\Local\Temp\tmpca_tpb35\model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into C:\Users\Surbhi\AppData\Local\Temp\tmpca_tpb35\model.ckpt.
INFO:tensorflow:step = 1001, loss = 2.02382e+11
INFO:tensorflow:global_step/sec: 366.546
INFO:tensorflow:step = 1101, loss = 3.24825e+11 (0.280 sec)
INFO:tensorflow:global_step/sec: 417.421
INFO:tensorflow:step = 1201, loss = 1.92516e+11 (0.237 sec)
INFO:tensorflow:global_step/sec: 405.943
INFO:tensorflow:step = 1301, loss = 5.48933e+11 (0.246 sec)
INFO:tensorflow:global_step/sec: 394.687
INFO:tensorflow:step = 1401, loss = 7.89179e+10 (0.254 sec)
INFO:tensorflow:global_step/sec: 412.709
INFO:tensorflow:step = 1501, loss = 2.15461e+11 (0.241 sec)
INFO:tensorflow:global_step/sec: 447.1
INFO:tensorflow:step = 1601, loss = 2.02085e+11 (0.228 sec)
INFO:tensorflow:global_step/sec: 315.47
INFO:tensorflow:step = 1701, loss = 6.19053e+10 (0.

INFO:tensorflow:step = 8901, loss = 6.41611e+10 (0.222 sec)
INFO:tensorflow:global_step/sec: 432.382
INFO:tensorflow:step = 9001, loss = 9.88495e+10 (0.232 sec)
INFO:tensorflow:global_step/sec: 414.194
INFO:tensorflow:step = 9101, loss = 9.94798e+10 (0.238 sec)
INFO:tensorflow:global_step/sec: 410.815
INFO:tensorflow:step = 9201, loss = 1.16101e+11 (0.246 sec)
INFO:tensorflow:global_step/sec: 390.479
INFO:tensorflow:step = 9301, loss = 6.63882e+10 (0.250 sec)
INFO:tensorflow:global_step/sec: 399.449
INFO:tensorflow:step = 9401, loss = 1.21386e+11 (0.250 sec)
INFO:tensorflow:global_step/sec: 396.443
INFO:tensorflow:step = 9501, loss = 9.55927e+10 (0.256 sec)
INFO:tensorflow:global_step/sec: 442.235
INFO:tensorflow:step = 9601, loss = 1.02383e+11 (0.229 sec)
INFO:tensorflow:global_step/sec: 396.823
INFO:tensorflow:step = 9701, loss = 2.62001e+10 (0.249 sec)
INFO:tensorflow:global_step/sec: 411.085
INFO:tensorflow:step = 9801, loss = 3.86436e+10 (0.242 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 444.084
INFO:tensorflow:step = 17001, loss = 3.99903e+10 (0.226 sec)
INFO:tensorflow:global_step/sec: 398.302
INFO:tensorflow:step = 17101, loss = 6.4418e+10 (0.247 sec)
INFO:tensorflow:global_step/sec: 403.335
INFO:tensorflow:step = 17201, loss = 4.03872e+10 (0.249 sec)
INFO:tensorflow:global_step/sec: 403.504
INFO:tensorflow:step = 17301, loss = 4.20179e+10 (0.248 sec)
INFO:tensorflow:global_step/sec: 409.347
INFO:tensorflow:step = 17401, loss = 1.48835e+11 (0.243 sec)
INFO:tensorflow:global_step/sec: 416.333
INFO:tensorflow:step = 17501, loss = 4.06189e+10 (0.242 sec)
INFO:tensorflow:global_step/sec: 392.252
INFO:tensorflow:step = 17601, loss = 1.02029e+11 (0.253 sec)
INFO:tensorflow:global_step/sec: 416.659
INFO:tensorflow:step = 17701, loss = 1.49734e+11 (0.242 sec)
INFO:tensorflow:global_step/sec: 393.198
INFO:tensorflow:step = 17801, loss = 5.42126e+10 (0.250 sec)
INFO:tensorflow:global_step/sec: 410.035
INFO:tensorflow:step = 17901, loss = 1.272

INFO:tensorflow:global_step/sec: 445.91
INFO:tensorflow:step = 25101, loss = 6.24566e+10 (0.221 sec)
INFO:tensorflow:global_step/sec: 436.66
INFO:tensorflow:step = 25201, loss = 1.08754e+11 (0.232 sec)
INFO:tensorflow:global_step/sec: 398.487
INFO:tensorflow:step = 25301, loss = 6.19023e+10 (0.247 sec)
INFO:tensorflow:global_step/sec: 413.82
INFO:tensorflow:step = 25401, loss = 4.04609e+10 (0.246 sec)
INFO:tensorflow:global_step/sec: 369.813
INFO:tensorflow:step = 25501, loss = 8.02094e+10 (0.266 sec)
INFO:tensorflow:global_step/sec: 400.364
INFO:tensorflow:step = 25601, loss = 6.6748e+10 (0.254 sec)
INFO:tensorflow:global_step/sec: 407.861
INFO:tensorflow:step = 25701, loss = 5.20187e+10 (0.242 sec)
INFO:tensorflow:global_step/sec: 411.783
INFO:tensorflow:step = 25801, loss = 9.26533e+10 (0.243 sec)
INFO:tensorflow:global_step/sec: 409.622
INFO:tensorflow:step = 25901, loss = 6.22725e+10 (0.248 sec)
INFO:tensorflow:Saving checkpoints for 26000 into C:\Users\Surbhi\AppData\Local\Temp\t

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [121]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(X_test,batch_size=10,num_epochs=1,shuffle=False)

In [122]:
prediction = list(model.predict(pred_input_func))

INFO:tensorflow:Restoring parameters from C:\Users\Surbhi\AppData\Local\Temp\tmpca_tpb35\model.ckpt-26000


** Calculate the RMSE **

In [123]:
pred = []
for each_pred in prediction:
    pred.append(each_pred['predictions'])


In [124]:
from sklearn.metrics import mean_squared_error

In [125]:
error = mean_squared_error(y_test,pred)**0.5

In [126]:
error

97652.299514246915